In [1]:
import chess
import chess.pgn

def get_fen_before_after(game):
    board = game.board()
    positions = []

    for move in game.mainline_moves():
        fen_before = board.fen()  # Get FEN before the move
        board.push(move)
        fen_after = board.fen()  # Get FEN after the move
        positions.append((fen_before, move, fen_after))

    return positions

def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

# Load games from PGN file
pgn_file_path = "analysis_pgns/0KNdpNQd---1910 Lasker vs. Schlechter.pgn"
all_games = load_games_from_pgn(pgn_file_path)

# Analyze games
for game in all_games:
    positions = get_fen_before_after(game)
    for fen_before, move, fen_after in positions:
        print(f"Move: {move}")
        print(f"FEN Before: {fen_before}")
        print(f"FEN After: {fen_after}")
        print()


Move: e2e4
FEN Before: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
FEN After: rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1

Move: e7e5
FEN Before: rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1
FEN After: rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2

Move: g1f3
FEN Before: rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
FEN After: rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2

Move: b8c6
FEN Before: rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2
FEN After: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3

Move: f1b5
FEN Before: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3
FEN After: r1bqkbnr/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/RNBQK2R b KQkq - 3 3

Move: g8f6
FEN Before: r1bqkbnr/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/RNBQK2R b KQkq - 3 3
FEN After: r1bqkb1r/pppp1ppp/2n2n2/1B2p3/4P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 4 4

Move: e1g1
FEN Before: r1bqkb1r/pppp1

In [2]:
# Import required libraries
import IPython.display as display
import ipywidgets as widgets
import os

# Define the saved output
saved_output = """
Event: World Championship Match 1937
Position: 8/7p/1rp5/3k4/2R2PP1/4K3/7P/8 w - - 1 42
Blunder: c4d4
Stockfish Evaluation: -3.04
Stockfish Suggested Move: c4d4
Maia 1100 Evaluation: 1.2, Suggested Move: c4d4
Maia 1200 Evaluation: 0.81, Suggested Move: c4d4
Maia 1300 Evaluation: 1.76, Suggested Move: c4d4
Maia 1400 Evaluation: 1.22, Suggested Move: c4d4
Maia 1500 Evaluation: 1.11, Suggested Move: c4d4
Maia 1600 Evaluation: 2.51, Suggested Move: c4d4
Maia 1700 Evaluation: 1.72, Suggested Move: c4d4
Maia 1800 Evaluation: 2.74, Suggested Move: c4d4
Maia 1900 Evaluation: 2.31, Suggested Move: c4d4
...
"""

# Function to parse the saved output
def parse_saved_output(output):
    events = []
    lines = output.strip().split("\n")
    current_event = None
    current_blunders = []

    for line in lines:
        if line.startswith("Event:"):
            if current_event:
                events.append((current_event, current_blunders))
                current_blunders = []
            current_event = line.split("Event:")[1].strip()
        elif line.startswith("Position:"):
            position = line.split("Position:")[1].strip()
        elif line.startswith("Blunder:"):
            blunder = line.split("Blunder:")[1].strip()
        elif line.startswith("Stockfish Evaluation:"):
            stockfish_eval = float(line.split("Stockfish Evaluation:")[1].strip())
        elif line.startswith("Stockfish Suggested Move:"):
            stockfish_best_move = line.split("Stockfish Suggested Move:")[1].strip()
        elif line.startswith("Maia"):
            parts = line.split()
            elo = int(parts[1])
            maia_eval = float(parts[3].strip(","))
            maia_best_move = parts[6]
            if len(current_blunders) == 0 or current_blunders[-1][:3] != (position, blunder, stockfish_eval):
                current_blunders.append((position, blunder, stockfish_eval, stockfish_best_move, {elo: (maia_best_move, maia_eval)}))
            else:
                current_blunders[-1][-1][elo] = (maia_best_move, maia_eval)
        elif line.startswith("Initial Image:") or line.startswith("Blunder Image:") or line.startswith("Correct Move Image:"):
            image_type, image_path = line.split(": ")
            current_blunders[-1].append(image_path.strip())

    if current_event:
        events.append((current_event, current_blunders))

    return events

# Parse the saved output
all_blunders = parse_saved_output(saved_output)

# Function to display images
def display_image(image_path):
    display.display(display.Image(image_path))

# Function to create an interactive blunder quiz
def blunder_quiz(event_index, blunder_index):
    event, blunder_list = all_blunders[event_index]
    blunder = blunder_list[blunder_index]

    fen, move, stockfish_eval, stockfish_best_move, maia_suggestions, initial_image_path, blunder_image_path, correct_image_path = blunder

    display.Markdown(f"## Blunder Quiz for {event}")
    display.Markdown(f"**Position (Before Blunder)**: {fen}")
    display_image(initial_image_path)

    # Create a dropdown for user to select the move
    move_options = [str(move)] + [str(stockfish_best_move)] + [str(best_move) for elo, (best_move, _) in maia_suggestions.items() if elo > 1300]
    move_dropdown = widgets.Dropdown(options=move_options, description='Your Move:')

    def check_answer(b):
        user_move = move_dropdown.value
        if user_move == str(stockfish_best_move):
            display.Markdown("### Correct Move!")
            display_image(correct_image_path)
        else:
            display.Markdown("### Blunder!")
            display_image(blunder_image_path)

    check_button = widgets.Button(description="Check Move")
    check_button.on_click(check_answer)

    display.display(move_dropdown)
    display.display(check_button)

# Create an interactive quiz for the first event and first blunder
blunder_quiz(0, 0)

# You can add more quizzes for other blunders by calling blunder_quiz with different indices

ValueError: not enough values to unpack (expected 8, got 5)